In [9]:
from sys import platform as _platform
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
#if _platform =='linux2':
#    path = '../data/data_sleep/' 
#else:
#    #mets ton path ici et ça devrait marcher :)
#    path = ""
path = "data/"

In [11]:
#frequencies=pd.read_csv(path+"data_frequences.csv").drop("Unnamed: 0", axis=1)
frequencies=pd.read_csv(path+"fft_eeg.csv")
frequencies_acc =pd.read_csv(path+"fft_acc.csv")

stats_acc=pd.read_csv(path+"stats_acc.csv")
stats_egg=pd.read_csv(path+"stats_eeg.csv")
stats=pd.concat([stats_acc, stats_egg], axis=1)
#ACC_feats =pd.read_csv(path+"ACC_feats.csv").drop("Unnamed: 0", axis=1)
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")
#stats = pd.concat([stats.filter(regex="EEG[0-9]*"),stats_ACC], axis=1)
#stats = pd.concat([stats,stats_ACC], axis=1)

frequencies_test=pd.read_csv(path+"fft_eeg_test.csv")
frequencies_acc_test =pd.read_csv(path+"fft_acc_test.csv")
#ACC_feats_test =pd.read_csv(path+"ACC_feats_test.csv").drop("Unnamed: 0", axis=1)
stats_acc_test=pd.read_csv(path+"stats_acc_test.csv")
stats_egg_test=pd.read_csv(path+"stats_eeg_test.csv")
stats_test=pd.concat([stats_acc_test, stats_egg_test], axis=1)

In [12]:
def select_freq_names(low, high, X_columns, prefix = ''):
    return [name for name in X_columns 
            if len(name.split('q'))==2 
            and name.split('freq')[0] == prefix
            and low<=float(name.split('freq')[1]) 
            and high>= float(name.split('freq')[1])]
def group_frequencies(name, low, high, frequencies, prefix = ''):
    frequencies[name]=(1./(high-low) * (frequencies[select_freq_names(low,high,frequencies.columns,prefix)])).sum(axis=1)

In [13]:

#frequencies["delta"]=frequencies[select_freq_names(0,3,frequencies.columns)].sum(axis=1)
#frequencies["delta"]=frequencies[select_freq_names(0,3.99,frequencies.columns)].sum(axis=1)
#frequencies["theta"]=frequencies[select_freq_names(4,7.5,frequencies.columns)].sum(axis=1)
#frequencies["alpha"]=frequencies[select_freq_names(7.5,13.99,frequencies.columns)].sum(axis=1)
#frequencies["beta"]=frequencies[select_freq_names(14,50,frequencies.columns)].sum(axis=1)

def make_new_feats(frequencies):
    group_frequencies("delta1", 0.4, 0.8, frequencies)
    group_frequencies("delta2", 0.81, 1.99, frequencies)
    group_frequencies("delta3", 2, 3.99, frequencies)
    group_frequencies("theta1", 4, 5, frequencies)
    group_frequencies("theta2", 5, 7.499, frequencies)
    group_frequencies("alpha1", 7.5, 9.5, frequencies)
    group_frequencies("alpha2", 9.5, 10.499, frequencies)
    group_frequencies("alpha3", 10.5, 13.5, frequencies)
    group_frequencies("beta", 14, 20, frequencies)
    group_frequencies("beta2", 20, 30, frequencies)
    group_frequencies("beta3", 31, 50, frequencies)



In [14]:
def regroup_acc_freq (frequencies_acc):
    for prefix in ['ACC_X.','ACC_Y.','ACC_Z.']:
        group_frequencies(prefix+"smaller_one",0.01,1, frequencies_acc,prefix)
        group_frequencies(prefix+"one_to_two",1.01,2, frequencies_acc,prefix)
        group_frequencies(prefix+"two_to_three",2.01,3, frequencies_acc,prefix)
        group_frequencies(prefix+"three_to_four",3.01,4, frequencies_acc,prefix)
        group_frequencies(prefix+"more_four",4,10, frequencies_acc,prefix)




In [15]:
def make_new_features(f, stat, f_acc):
    new_feat = ["delta1","delta2","delta3", 'theta1', 'theta2', 'alpha1','alpha2','alpha3', 'beta','beta2','beta3']
    make_new_feats(f)
    
    prefixes = ['ACC_X.','ACC_Y.','ACC_Z.']
    regroup_acc_freq (f_acc)
    frequencies_acc2 = f_acc[[prefix+ x for x in["smaller_one","one_to_two","two_to_three",'more_four']for prefix in prefixes]]
    
    return  pd.concat([f[new_feat], stat, frequencies_acc2], axis=1)

In [16]:
train = make_new_features(frequencies, stats, frequencies_acc)
test = make_new_features(frequencies_test, stats_test, frequencies_acc_test)

MemoryError: 

In [ ]:
train.describe()

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
y_train = labels['TARGET']
X = train.drop(["skew_ACC_X","skew_ACC_Y", "skew_ACC_Z"], axis=1)
X_t = test.drop(["skew_ACC_X","skew_ACC_Y", "skew_ACC_Z"], axis=1)
#X, X_t, y_train, y_test = train_test_split(
#    X, labels["TARGET"], test_size=0.2)

scaler = StandardScaler().partial_fit(X)
scaler = StandardScaler().partial_fit(X_t)
X_train = scaler.transform(X)
X_test = scaler.transform(X_t)
X_columns = train.columns

In [ ]:
%%time
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import cohen_kappa_score
et = ExtraTreesClassifier(random_state = 42,n_estimators=1000, max_depth= 29, n_jobs = 3, class_weight = 'balanced')
et.fit(X_train, y_train)
predicted_label = et.predict(X_test)

#print("Extra Random Trees - accuracy Score on test_data : ", accuracy_score(y_test, predicted_label))
#print("Extra Random Trees - kappa Score on test_data : ", cohen_kappa_score(y_test, predicted_label))
print("Extra Random Trees - kappa Score on train data : ", cohen_kappa_score(y_train, et.predict(X_train)))

In [ ]:
#from sklearn.metrics import accuracy_score
#print("Extra Random Trees - accuracy Score on test_data : ", accuracy_score(y_test, predicted_label))
#print("Extra Random Trees - kappa Score on test_data : ", cohen_kappa_score(y_test, predicted_label))

In [ ]:
plt.figure(figsize=(15, 5))

ordering = np.argsort(et.feature_importances_)[::-1][:50]

importances = et.feature_importances_[ordering]
feature_names = X_columns[ordering]

x = np.arange(len(feature_names))
plt.bar(x, importances)
plt.xticks(x + 0.5, feature_names, rotation=90, fontsize=15);

In [13]:
%%time
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(et, X_train, y_train, cv=5, scoring="accuracy",n_jobs=3)
print("accuracy: {:.4f} +/-{:.4f}".format(
    np.mean(scores), np.std(scores)))

accuracy: 0.8977 +/-0.0026
CPU times: user 189 ms, sys: 179 ms, total: 368 ms
Wall time: 2min 29s


In [14]:
def make_submission(predicted_label, name = 'submit.csv'):
    ids = pd.read_csv('../data/data_sleep/input_test.csv', usecols= ['ID'])
    ids["TARGET"] = pd.Series(predicted_label)
    ids['TARGET']= ids['TARGET'].astype(int)
    ids.to_csv(name,index=False)

In [15]:
make_submission(predicted_label)

In [16]:
predicted_label_train = et.predict(X_train)

In [17]:
import collections
print collections.Counter(predicted_label), collections.Counter(y_train)

Counter({2.0: 26010, 4.0: 2276, 3.0: 1598, 0.0: 493, 1.0: 81}) Counter({2.0: 15334, 3.0: 9640, 4.0: 4385, 0.0: 1342, 1.0: 428})


In [18]:
X_train.shape, X_test.shape

((31129, 81), (30458, 81))